In [0]:
import nltk
import numpy as np
import pandas as pd

In [0]:
train = pd.read_csv(r'C:\Users\asus\Downloads\TwitterSentiment\train.csv')
test = pd.read_csv(r'C:\Users\asus\Downloads\TwitterSentiment\test.csv')

In [0]:
train_data = train.text
train_target = train.airline_sentiment
test_data = test.text

In [0]:
from nltk.corpus import stopwords
import string
stop = stopwords.words('english')
stop.remove("isn't")
punctuations = list(string.punctuation)
stop += punctuations

In [0]:
from nltk.tokenize import word_tokenize, TweetTokenizer
#nltk.download('punkt')

In [0]:
# s = "yes..not"
tt = TweetTokenizer(strip_handles=False)
# word_tokenize(s), tt.tokenize(s)

In [0]:
documents = []
for i in range(len(train_data)):
    documents.append([train_data[i], train_target[i]])

In [0]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import wordnet
lemmatizer = WordNetLemmatizer()

In [0]:
def postag(st):
    s = str(st)
    if s.startswith('J'):
        return wordnet.ADJ
    elif s.startswith('N'):
        return wordnet.NOUN
    elif s.startswith('V'):
        return wordnet.VERB
    elif s.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [0]:
def clean_words(doc):
    output_words = []
    words = tt.tokenize(doc)
    for word in words:
        if (word.lower() not in stop) and (word.isdigit() == False):
            pos = pos_tag(word)
            out = lemmatizer.lemmatize(word, pos = postag(pos))
            output_words.append(out)
    return output_words

In [0]:
documents_new = [(clean_words(doc), target) for doc, target in documents]

In [0]:
testing_data = [clean_words(doc) for doc in test_data]

In [0]:
# all_words = []
# for doc, tar in documents_new:
#     all_words += doc

In [0]:
# import nltk
# freq = nltk.FreqDist(all_words)
# common_words = freq.most_common(3000)
# vocab = [word for word, val in common_words]

In [0]:
# def features(doc):
#     feature = {}
#     for w in vocab:
#         feature[w] = w in doc
#     return feature

In [0]:
# training_data = [(features(doc), target) for doc, target in documents_new]

In [0]:
# clf = nltk.NaiveBayesClassifier.train(training_data)

In [0]:
# clf.show_most_informative_features(1000)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

In [0]:
x_train = [" ".join(doc) for doc, target in documents_new]
y_train = [target for doc, target in documents_new]
x_test = [" ".join(data) for data in testing_data]

In [0]:
x_tr, x_te, y_tr, y_te = train_test_split(x_train, y_train, random_state = 1)

In [0]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [0]:
Count_Vec = CountVectorizer(max_features= 2890, max_df=0.6, tokenizer= tt.tokenize)#, ngram_range=(1,2))
x_train_v = Count_Vec.fit_transform(x_train)
#x_tr_v = Count_Vec.fit_transform(x_tr)

In [0]:
# Count_Vec.get_feature_names()

In [0]:
x_test_v = Count_Vec.transform(x_test)
#x_te_v = Count_Vec.transform(x_te)

In [0]:
#svc = SVC()
rfc = RandomForestClassifier()
Multi = MultinomialNB(alpha=0.27)
#svc.fit(x_train_v, y_train)

In [0]:
#Multi.fit(x_tr_v, y_tr)

In [0]:
#Multi.score(x_te_v, y_te)

In [0]:
from sklearn.model_selection import cross_val_score, KFold

In [0]:
# cv = KFold(n_splits= 4, shuffle= False, random_state= 1)
# cross_val_score(Multi, x_train_v, y_train, scoring = 'accuracy', cv = cv, verbose = 1)

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished


array([0.771949  , 0.76065574, 0.76211293, 0.77340619])

In [0]:
Multi.fit(x_train_v, y_train)

MultinomialNB(alpha=0.27, class_prior=None, fit_prior=True)

In [0]:
y_pred = Multi.predict(x_test_v)

In [0]:
np.savetxt(r'C:\Users\asus\Downloads\TwitterSentiment\ypreddcop.csv', y_pred, delimiter=',', fmt="%s")